# Get started with Computer Vision and TF

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/Rishit-dagli/Design-and-Code-2020/blob/master/cats-vs-dogs.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/Rishit-dagli/Design-and-Code-2020/blob/master/cats-vs-dogs.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://github.com/Rishit-dagli/Design-and-Code-2020/blob/master/cats-vs-dogs.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this notebook you will create your first Computer Vision based Deep Learning model to classify between cats and dogs with TensorFlow. We will use the [dogs-vs-cats dataset](https://www.kaggle.com/c/dogs-vs-cats/data) which is open-sourced. We will create a simple 2 class binary CNN to do so. You wil see all the aspects of training such a model, data preprocressing, the actual model, reducing overfitting and finally making inferences.

This guide uses [tf.keras](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/keras.ipynb), a high-level API to build and train models in TensorFlow.

> Note: I have marked section with `Only in Colab` and `Only in Local` to make this notebook useable by all

|Cats|Dogs|
|----|----|
|![](http://storage.rishit.tech/storage/Design-And-Code-2020/sample-cats.png)|![](http://storage.rishit.tech/storage/Design-And-Code-2020/sample-dogs.png)|

## Setup

### Download data (Only in Colab)

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /cats_and_dogs_filtered.zip

### Imports

In [5]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_HEIGHT = IMG_WIDTH = 150

local_zip = 'cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/')
zip_ref.close()

## Specifying directories

We will now specify our training and testing directories so things become easier for us at later stages.

In [6]:
base_dir = '/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')